# Set up `rxmx.walker.Walker`

In [1]:
import lzma
from pathlib import Path

import dill as pickle
import numpy as np
from tqdm import tqdm

In [2]:
from scipy import stats
from scipy.optimize import minimize

In [3]:
import elm

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


In [4]:
import rxmc

In [5]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
likelihood_model_dir = Path("../likelihood_model/")
prior_path = Path("../prior/prior_distribution.pickle")

## read in ELM prior parameter samples

In [6]:
with open(prior_path, "rb") as f:
    prior = pickle.load(f)
prior_samples = prior.rvs(1000)

## read in observations

In [7]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

## read in physical models

In [8]:
with open(model_dir / "elm_nn_model.pkl", "rb") as f:
    elm_nn_model = pickle.load(f)

In [9]:
with open(model_dir / "elm_pp_model.pkl", "rb") as f:
    elm_pp_model = pickle.load(f)

## read in likelihood models

In [10]:
with open(likelihood_model_dir / "likelihood_model.pkl", "rb") as f:
    likelihood_model = pickle.load(f)

## Set up Evidence object

In [11]:
pp_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=elm_pp_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in pp_elastic_obs.items()
]
nn_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=elm_nn_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in nn_elastic_obs.items()
]

In [12]:
evidence = rxmc.evidence.Evidence(constraints=nn_constraints + pp_constraints)

In [13]:
with open("../mlem/mlem_result.pkl", "rb") as f:
    starting_loc = pickle.load(f).x

In [14]:
starting_loc

array([ 5.85559187e+01,  1.31285844e+01,  2.74603112e+01,  2.97387385e+01,
        1.89849500e+01,  3.14454870e+01, -4.00000000e-01,  1.20519866e-03,
        5.30123070e+01,  2.23023592e+01,  1.17726497e+00,  1.39280797e+00,
        6.82664567e-01,  4.00000000e-01])

In [15]:
rng = np.random.default_rng(13)

In [16]:
proposal = rxmc.proposal.NormalProposalDistribution(
    cov=np.diag((np.diag(prior.cov) * 0.01**2)),
)

In [17]:
sampling_config = rxmc.param_sampling.SamplingConfig(
    starting_location=starting_loc,
    params=elm.elm.params,
    proposal=proposal,
    prior=prior,
)

In [18]:
walker = rxmc.walker.Walker(
    model_sample_conf=sampling_config,
    evidence=evidence,
    rng=rng,
)

In [19]:
with lzma.open("./walker.xz", "wb") as f:
    pickle.dump(walker, f)